In [9]:
%pip install lxml
import os
import s3fs
import gzip
import xml.etree.ElementTree as ET
from lxml import etree
import pandas as pd

os.environ["AWS_ACCESS_KEY_ID"] = 'DRSDSRMN9R1N9B2FBDBI'
os.environ["AWS_SECRET_ACCESS_KEY"] = '+6bqWKjLlYIUU79tXFsXnZslT6N7dKNQoHN+E7Hc'
os.environ["AWS_SESSION_TOKEN"] = 'eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJhY2Nlc3NLZXkiOiJEUlNEU1JNTjlSMU45QjJGQkRCSSIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sImF1ZCI6WyJtaW5pby1kYXRhbm9kZSIsIm9ueXhpYSIsImFjY291bnQiXSwiYXV0aF90aW1lIjoxNzYwNTExNzk3LCJhenAiOiJvbnl4aWEiLCJlbWFpbCI6Imx1Y2FzLmN1bXVuZWxAZW5zYWUuZnIiLCJlbWFpbF92ZXJpZmllZCI6dHJ1ZSwiZXhwIjoxNzYxNjM0OTc5LCJmYW1pbHlfbmFtZSI6IkN1bXVuZWwiLCJnaXZlbl9uYW1lIjoiTHVjYXMiLCJncm91cHMiOlsiVVNFUl9PTllYSUEiLCJzdGF0YXBwLXNlZ21lZGljIl0sImlhdCI6MTc2MTAzMDE3OSwiaXNzIjoiaHR0cHM6Ly9hdXRoLmxhYi5zc3BjbG91ZC5mci9hdXRoL3JlYWxtcy9zc3BjbG91ZCIsImp0aSI6Im9ucnRydDo3OTg2NTM3OC0wMzZmLTU5ZWYtMjRjYy04YWQxM2Q5M2QxZTciLCJuYW1lIjoiTHVjYXMgQ3VtdW5lbCIsInBvbGljeSI6InN0c29ubHkiLCJwcmVmZXJyZWRfdXNlcm5hbWUiOiJsYWIiLCJyZWFsbV9hY2Nlc3MiOnsicm9sZXMiOlsib2ZmbGluZV9hY2Nlc3MiLCJ1bWFfYXV0aG9yaXphdGlvbiIsImRlZmF1bHQtcm9sZXMtc3NwY2xvdWQiXX0sInJlc291cmNlX2FjY2VzcyI6eyJhY2NvdW50Ijp7InJvbGVzIjpbIm1hbmFnZS1hY2NvdW50IiwibWFuYWdlLWFjY291bnQtbGlua3MiLCJ2aWV3LXByb2ZpbGUiXX19LCJyb2xlcyI6WyJvZmZsaW5lX2FjY2VzcyIsInVtYV9hdXRob3JpemF0aW9uIiwiZGVmYXVsdC1yb2xlcy1zc3BjbG91ZCJdLCJzY29wZSI6Im9wZW5pZCBwcm9maWxlIGdyb3VwcyBlbWFpbCIsInNpZCI6ImYxNzIyZjExLWZmMjMtNDFkOS04MTAzLWY3MTRhOGIwZTY5ZiIsInN1YiI6ImUyZDc4NjRjLTcwMzItNDI0ZC04OTA2LWU0ZjhiNDFjYzAwMyIsInR5cCI6IkJlYXJlciJ9.qxKq-xA7_3KcVex9J292-nuw8FtDsGF55CzOgJpNEDE03KAbgyDnkekSOFGsGE8Yhr_tMlBuXfLZGhQSY1chdw'
os.environ["AWS_DEFAULT_REGION"] = 'us-east-1'
fs = s3fs.S3FileSystem(
    client_kwargs={'endpoint_url': 'https://'+'minio.lab.sspcloud.fr'},
    key = os.environ["AWS_ACCESS_KEY_ID"], 
    secret = os.environ["AWS_SECRET_ACCESS_KEY"], 
    token = os.environ["AWS_SESSION_TOKEN"])

path = "lab/dblp (1).xml.gz"



Note: you may need to restart the kernel to use updated packages.


In [10]:
#names.to_parquet("conf_names.parquet", index=False)
s3_path = "s3://lab/jap_girls.csv"  # bucket + key

# Save directly to MinIO
girls_df.to_csv(s3_path, index=False)

PermissionError: The Access Key Id you provided does not exist in our records.

In [14]:

with fs.open("s3://lab/jap_boys.csv", "w") as f:
    boys_df.to_csv(f, index=False)

In [4]:
articles = []

with fs.open(path, 'rb') as f:
    with gzip.GzipFile(fileobj=f) as gz:
        #parser = etree.XMLParser(recover=True, resolve_entities=True)
        context = etree.iterparse(gz, events=("end",), tag="article", recover=True)
        
        for event, elem in context:
            title = elem.findtext("title")
            journal = elem.findtext("journal")
            pub_date = elem.findtext("year")

            authors = [a.text for a in elem.findall(".//author")]
            authors_str = "; ".join([a for a in authors if a])

            articles.append({
                "title": title,
                "journal": journal,
                "year": pub_date,
                "authors": authors_str
            })

            elem.clear()

df = pd.DataFrame(articles)

KeyboardInterrupt: 

In [2]:
inpro = []

with fs.open(path, 'rb') as f:
    with gzip.GzipFile(fileobj=f) as gz:
        #parser = etree.XMLParser(recover=True, resolve_entities=True)
        context = etree.iterparse(gz, events=("end",), tag="inproceedings", recover=True)
        
        for event, elem in context:
            title = elem.findtext("title")
            conf = elem.findtext("booktitle")
            pub_date = elem.findtext("year")

            authors = [a.text for a in elem.findall(".//author")]
            authors_str = "; ".join([a for a in authors if a])

            inpro.append({
                "title": title,
                "conf": conf,
                "year": pub_date,
                "authors": authors_str
            })

            elem.clear()

df = pd.DataFrame(inpro)

In [9]:

import re
df[df["authors"].apply(lambda x: bool(re.compile(r"^[A-Za-z\s-]+$").search(str(x))))]


,title,conf,year,authors
0,The Future of Classic Data Administration: Obj...,SWEE,1998,Arnon Rosenthal
1,Some Patterns of Convincing Software Engineeri...,Denert Award,2020,Lutz Prechelt
2,Static Worst-Case Analyses and Their Validatio...,Denert Award,2020,Peter Wgemann
3,Crossing Disciplinary Borders to Improve Requi...,Denert Award,2020,Anne Hess
4,What You See Is What You Get: Practical Effect...,Denert Award,2020,Jonathan Immanuel Brachthuser
...,...,...,...,...
3749504,"Machine Intentionality, the Moral Status of Ma...",PT-AI,2011,David Leech Anderson
3749509,Practical Introspection as Inspiration for AI.,PT-AI,2011,Sam Freed
3749510,"""Computational Ontology and Deontology"".",PT-AI,2011,Raffaela Giovagnoli
3749512,Emotional Control-Conditio Sine Qua Non for Ad...,PT-AI,2011,Claudius Gros


In [2]:
from lxml import etree
import gzip

direct_set = set()

with fs.open(path, "rb") as f:
    with gzip.GzipFile(fileobj=f) as gz:
        context = etree.iterparse(gz, events=("end",), tag="inproceedings", recover=True)
        for event, elem in context:
            # collect child tags
            for child in elem:
                if child.tag is not None:
                    direct_set.add(child.tag)

            # clear memory: remove element and its previous siblings
            elem.clear()
            while elem.getprevious() is not None:
                del elem.getparent()[0]

print(sorted(direct_set))

['author', 'booktitle', 'cdrom', 'cite', 'crossref', 'editor', 'ee', 'month', 'note', 'number', 'pages', 'publnr', 'stream', 'title', 'url', 'volume', 'year']


In [3]:
chunks = []
n_chunks = 50
chunk_size = len(df) // n_chunks

for i in range(n_chunks):
    start = i * chunk_size
    end = (i + 1) * chunk_size if i < n_chunks - 1 else len(df)  # last chunk includes remainder
    
    chunk = df.iloc[start:end].copy()
    
    # Split authors column into multiple columns
    #authors_df = chunk['authors'].str.split(';', expand=True)
    #authors_df.columns = [f'author{i+1}' for i in range(authors_df.shape[1])]
    
    # Combine with original chunk
    #chunk = pd.concat([chunk.drop(columns=['authors']), authors_df], axis=1)
    
    chunk.to_parquet(f"inpro/inproceedings_chunk_{i}.parquet", index=False)

In [16]:

import pandas as pd
import re
from collections import defaultdict
import glob

ego_data = defaultdict(lambda: {"publications": []}) #set permet de créer des clefs si absentes et de les actualiser si présentes (en définissant la classe des clefs)

chunk_files = glob.glob("art/articles_chunk_*.parquet")

for file in chunk_files:
    df_chunk = pd.read_parquet(file)
    for _, row in df_chunk.iterrows():
        authors = row['authors'].split(';')
        authors = [a for a in authors if not re.compile(r"\d").search(a)]
        for ego in authors:
            coauthors = set(authors) - {ego}
            
            ego_data[ego]["publications"].append({
                "coauthors": list(coauthors),
                "journal": row.get("journal"),
                "year": row.get("year"),
                "title": row.get("title")
            })

In [24]:
"""ego_df = pd.DataFrame([
    {'ego': ego, **data["publications"]} #récup paries ego - alter et trie dans l'ordre alphabétique
    for ego, data in ego_data.items()
])"""
rows = []
for ego, data in ego_data.items():
    for pub in data["publications"]:
        rows.append({
            "ego": ego,
            "coauthors": ",".join(sorted(pub["coauthors"])),
            "journal": pub.get("journal"),
            "year": pub.get("year"),
            "title": pub.get("title")
        })

ego_df = pd.DataFrame(rows)


In [27]:
import numpy as np
ego_df["name"]=np.where(len(ego_df["ego"].str.split())>1, ego_df["ego"].str.split(n=1).str[0], None)
def is_valid_firstname(word):
    if not isinstance(word, str):  # skip NaN/None
        return False
    if len(word) == 1:  # single letter
        return False
    if word.isupper():  # all caps (acronyms)
        return False
    if not re.match(r'^[A-Za-z-]+$', word):  # contains only letters or hyphens
        return False
    return True

ego_df["name"] = ego_df["name"].apply(lambda x: x if is_valid_firstname(x) else None)


In [28]:
names=ego_df["name"].str.lower()


In [31]:
(names=="").sum()

np.int64(0)

In [32]:
names=names.drop_duplicates()

In [33]:
#names.to_parquet("conf_names.parquet", index=False)
s3_path = "s3://lab/art_names.csv"  # bucket + key

# Save directly to MinIO
names.to_csv(s3_path, index=False)

In [8]:
ego_df.to_parquet("ego_df.parquet",index=False)


In [19]:
with fs.open("s3://lab/name_gender_dataset.csv") as f:
    g1 = pd.read_csv(f)
with fs.open("s3://lab/wgnd_2_0_name-gender-langcode.csv") as f:
    g2 = pd.read_csv(f)

g1=g1[["Name","Gender"]]
g2[["Name","Gender"]]=g2[["name","gender"]]
g2=g2[["Name","Gender"]]

In [20]:
g2[100:150]

,Name,Gender
100,a laura,F
101,a lelia,F
102,a lelia,F
103,a lerah,F
104,a lerah,F
105,a lex ander,M
106,a lex ander,M
107,a lie,M
108,a lisa,F
109,a lisa,F


In [22]:
pd.concat([g1,g2],axis=0,ignore_index=True)

,Name,Gender
0,James,M
1,John,M
2,Robert,M
3,Michael,M
4,William,M
...,...,...
21978307,히카리,F
21978308,히토미,F
21978309,힘찬,M
21978310,凉峰,M
